In [ ]:
import json
import glob
import os
import pandas as pd
from pathlib import Path
from llama_stack_provider_trustyai_garak.intents import generate_intents_from_dataset
from llama_stack_provider_trustyai_garak.utils import _ensure_xdg_vars

_ensure_xdg_vars()

# Dataset → Garak Typology & Intent Stubs

Converts a red-teaming dataset produced by an **sdg_hub** flow into the files expected by Garak

- `trait_typology.json` — one entry per harm category (`policy_concept` → intent)
- `intent_stubs/<ID>.txt` — one file per category, one attack prompt per line (`prompt` → intent stub)

## Setup

In [ ]:
xdg_data = os.environ["XDG_DATA_HOME"]

dataset_files = sorted(Path(xdg_data).glob("*.json"))
if not dataset_files:
    raise FileNotFoundError(f"No JSON files found in {xdg_data}")

dataset_path = dataset_files[-1]
print(f"Loading: {dataset_path}")

df = pd.read_json(dataset_path)
print(f"Shape: {df.shape}")
df.head(3)

## Load dataset

Pick the most recent JSON file from `$XDG_DATA_HOME` — the output of the sdg_hub red-teaming flow.

In [ ]:
print("Policy concepts (intents):", df["policy_concept"].unique().tolist())
df[["policy_concept", "prompt"]].head(10)

## Inspect

Check the harm categories (intents) and sample prompts before conversion.

In [ ]:
df_clean = df.dropna(subset=["prompt"])
print(f"Rows after dropping null prompts: {len(df_clean)} (dropped {len(df) - len(df_clean)})")

# TODO: generate_intents_from_dataset should handle this!
# Garak intent codes must match [CTMS][0-9]{3}[a-z]* — lowercase letters only,
# no spaces or special characters.  Normalize policy_concept before conversion.
df_clean = df_clean.copy()
df_clean["intent_key"] = (
    df_clean["policy_concept"].str.lower().str.replace(r"[^a-z]", "", regex=True)
)
print("Intent keys:", df_clean["intent_key"].unique().tolist())

# Actually generate the Garak data
generate_intents_from_dataset(
    df_clean,
    category_column_name="intent_key",
    prompt_column_name="prompt",
)
print("Done.")

## Convert

Drop rows with unparsed prompts (`NaN`), then call `generate_intents_from_dataset` to write the typology and stub files under `$XDG_DATA_HOME/garak/data/cas/`.

In [ ]:
xdg_data = os.environ.get("XDG_DATA_HOME", str(Path.home() / ".local" / "share"))
typology_path = Path(xdg_data) / "garak" / "data" / "cas" / "trait_typology.json"

with open(typology_path) as f:
    typology = json.load(f)

print(f"Typology written to: {typology_path}\n")
print(json.dumps(typology, indent=2))

## Verify output

### Trait typology

In [ ]:
stubs_dir = Path(xdg_data) / "garak" / "data" / "cas" / "intent_stubs"
stub_files = sorted(stubs_dir.glob("*.txt"))

print(f"Intent stub files in {stubs_dir}:\n")
for stub_file in stub_files:
    lines = stub_file.read_text().splitlines()
    print(f"  {stub_file.name} — {len(lines)} stubs")
    for line in lines[:2]:
        print(f"    • {line[:80]}...")
    print()